<span style="color:#FF7F50">Notebook used to detect if there is a **Sync_In** signal present in the radar data</span>

In [9]:
import csv
from empkins_io.datasets.d03.micro_gapvii._dataset import MicroBaseDataset
from pathlib import Path
import numpy as np

<span style="color:#FF7F50">First we rename all raw emrad .h5 files correctly</span>

In [60]:
import os
import fnmatch
import glob
import re
for raw_emrad_dir in glob.glob("C:/Users/clark/Documents/Studium/Bachelorarbeit/data/data_per_subject/VP_[0-9][0-9]/*/emrad/raw"):
    res_vp = re.findall("VP_(\d+)", raw_emrad_dir)
    res_con = [s.replace('\\', '') for s in re.findall(".tsst", raw_emrad_dir)]
    files = [file for file in os.listdir(raw_emrad_dir) if fnmatch.fnmatch(file, '*.h5')]
    new_file_name = 'emrad_data_vp_' + res_vp[0] + '_' + res_con[0] + '.h5'
    if (len(files) == 1):
        os.rename(os.path.join(raw_emrad_dir,files[0]), os.path.join(raw_emrad_dir, new_file_name))
    elif (len(files) > 1):
        os.rename(os.path.join(raw_emrad_dir, [s for s in files if "emrad" in s][0]), os.path.join(raw_emrad_dir, new_file_name))
    
#count = 0
#for root, dirs, files in os.walk(r"C:\Users\clark\Documents\Studium\Bachelorarbeit\data"):
#    for file in files:
#        if fnmatch.fnmatch(file, 'emrad_data_vp_*_*tsst.h5'):
#            count +=1
#            print(os.path.join(root, file))
#print(count)

<span style="color:#FF7F50">Now we do the check for the Sync_In peak in the emrad signal, printing the result</span>

In [58]:
def detected_radar_sync_peaks(): 

    ## Create CSV file for storing the results of the search
    #csv_header_string = ["Participant", "Condition", "Node", "Sync_In Peak found (y/n)"]

    data = MicroBaseDataset(Path(r"C:\Users\clark\Documents\Studium\Bachelorarbeit\data"))
    # group all data by participant and condition
    grouped_data = data.groupby(['subject', 'condition'])

    # loop over groups
    for row in grouped_data:
        
        if (row.group[0] in data.MISSING_DATA):
            continue

        radar_data, _ = row.emrad

        for node, new_df in radar_data.groupby(axis = 1, level=0):

            if len(np.argwhere(new_df[node]['Sync_In'].values == 1))==0:

                print("No Sync_In peak found for " + row.group[0] + ", " + row.group[1] + "!")
                break

    ###################
    #THE OUT-COMMENTED CODE FOLLOWING CAN BE USED TO WRITE THE RESULTS OF THIS CHECK INTO A CSV FILE
    ###################

    #with open("detected_sync_peaks.csv", 'w', newline='') as file:

    #    writer = csv.writer(file)
    #    writer.writerow(csv_header_string)

    #   data = MicroBaseDataset(Path(r"C:\Users\clark\Documents\Studium\Bachelorarbeit\data"))
    #    # group all data by participant and condition
    #    grouped_data = data.groupby(['subject', 'condition'])

    #    # loop over groups
    #    for row in grouped_data:
    #        if (row.group[0] in data.MISSING_DATA):
    #            continue
    #       # next entry specifying the participant and the condition
    #        new_entry = [row.group[0], row.group[1]]
    #        radar_data, _ = row.emrad
    #        for node, new_df in radar_data.groupby(axis = 1, level=0):
    #            # append the radar node to the new index
    #            new_node = new_entry.copy()
    #            new_node.append(node)
    #            # search for the sync peak and document the result
    #            new_node.append("y" if len(np.argwhere(new_df[node]['Sync_In'].values == 1))>0 else "n")

    #            # write the complete result to the csv file
    #            writer.writerow(new_node)

    #    file.close()

In [62]:
detected_radar_sync_peaks()

No Sync_In peak found for VP_02, tsst!
No Sync_In peak found for VP_02, ftsst!
No Sync_In peak found for VP_03, tsst!
No Sync_In peak found for VP_03, ftsst!
